In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
# #     for filename in filenames:
# #         print(os.path.join(dirname, ""))
#     print(dirname)
print("started")

started


In [2]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import layers, models
from keras.models import Sequential
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall, Accuracy

2024-01-22 17:01:42.603775: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-22 17:01:42.605280: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-22 17:01:42.637897: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-22 17:01:42.638861: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-22 17:01:43.472507: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
CLASS = 4
batch_size = 64
img_height = 64
img_width = 64
train_dir = os.path.join("./dataset/faceEmotion/train")
test_dir = os.path.join("./dataset/faceEmotion/test")
CLASS, batch_size, (img_height, img_width), train_dir, test_dir

(4, 64, (64, 64), './dataset/faceEmotion/train', './dataset/faceEmotion/test')

In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
#     color_mode="grayscale",
    label_mode="categorical",
    interpolation="area",
    validation_split=0.01,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)
train_ds, train_ds.class_names

Found 21107 files belonging to 4 classes.
Using 20896 files for training.


(<_BatchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>,
 ['fear', 'happy', 'neutral', 'sad'])

In [5]:
# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"), cmap="gray")
# #     plt.title(train_ds.class_names[labels[i]])
#     plt.title(train_ds.class_names[np.argmax(labels[i])])
# #     print(np.argmax(labels[i]))
#     plt.axis("off")

In [5]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
#     color_mode="grayscale",
    label_mode="categorical",
    interpolation="area",
    validation_split=0.99,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)
test_ds, test_ds.class_names

Found 5278 files belonging to 4 classes.
Using 5225 files for validation.


(<_BatchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>,
 ['fear', 'happy', 'neutral', 'sad'])

In [7]:
# plt.figure(figsize=(10, 10))
# for images, labels in test_ds.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"), cmap="gray")
# #     plt.title(train_ds.class_names[labels[i]])
#     plt.title(test_ds.class_names[np.argmax(labels[i])])
# #     print(np.argmax(labels[i]))
#     plt.axis("off")

In [6]:
from tensorflow.keras.layers.experimental import preprocessing
from keras.layers import Dense, Conv2D, Flatten,ReLU , MaxPooling2D, Rescaling, Reshape, Dropout, BatchNormalization, AveragePooling2D, Input, RandomFlip, RandomRotation, Concatenate

In [7]:
def residual_block(x, filters, strides=1):
    # Shortcut branch
    shortcut = x

    # Main branch
    x = layers.Conv2D(filters, kernel_size=(3, 3), strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Add shortcut branch if strides are different
    if strides != 1 or filters != shortcut.shape[-1]:
        shortcut = layers.Conv2D(filters, kernel_size=(1, 1), strides=strides, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)
    return x

inputs = layers.Input(shape=(img_height, img_width, 3))  # Define the input layer
x = inputs  # Assign the input to x
x = preprocessing.RandomFlip('horizontal') (x) # flip left-to-right
x = preprocessing.RandomWidth(factor=0.2) (x) # horizontal stretch
x = preprocessing.RandomRotation(factor=0.50) (x)
x = preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.50) (x)
x = preprocessing.RandomContrast(0.3) (x)  # contrast change by up to 50%
x = layers.Rescaling(1./255) (x)

# Initial conv layers (modified for smaller input)
x = layers.Conv2D(10, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)  # Reduced kernel size and filters
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

# Stack residual blocks (adjust filter counts as needed)
x = residual_block(x, filters=32, strides=1)
x = residual_block(x, filters=32, strides=1)
x = residual_block(x, filters=64, strides=2)
x = residual_block(x, filters=64, strides=1)
x = residual_block(x, filters=128, strides=2)
x = residual_block(x, filters=128, strides=1)
x = residual_block(x, filters=256, strides=2)
x = residual_block(x, filters=256, strides=1)

# Final layers
x = layers.GlobalAveragePooling2D()(x)

outputs = layers.Dense(CLASS, activation='softmax')(x)  # Adjust output neurons for your task

model = models.Model(inputs=inputs, outputs=outputs)  # Create the model

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy', Precision(), Recall()],
)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 random_flip (RandomFlip)    (None, 64, 64, 3)            0         ['input_1[0][0]']             
                                                                                                  
 random_width (RandomWidth)  (None, 64, None, 3)          0         ['random_flip[0][0]']         
                                                                                                  
 random_rotation (RandomRot  (None, 64, None, 3)          0         ['random_width[0][0]']        
 ation)                                                                                       

In [8]:
epochs=260
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=epochs,
    callbacks=[EarlyStopping(monitor='val_accuracy', patience=epochs/10, min_delta=0.001, mode='max', restore_best_weights=True)]
)

Epoch 1/260


327/327 [==============================] - 106s 310ms/step - loss: 1.3995 - accuracy: 0.3226 - precision: 0.3565 - recall: 0.0231 - val_loss: 1.4023 - val_accuracy: 0.3278 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/260
327/327 [==============================] - 102s 311ms/step - loss: 1.3609 - accuracy: 0.3333 - precision: 0.3730 - recall: 0.0078 - val_loss: 1.5440 - val_accuracy: 0.1964 - val_precision: 0.2671 - val_recall: 0.0794
Epoch 3/260
327/327 [==============================] - 99s 304ms/step - loss: 1.3595 - accuracy: 0.3361 - precision: 0.4056 - recall: 0.0174 - val_loss: 1.4288 - val_accuracy: 0.2488 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/260
327/327 [==============================] - 99s 303ms/step - loss: 1.3502 - accuracy: 0.3461 - precision: 0.4533 - recall: 0.0144 - val_loss: 1.3788 - val_accuracy: 0.2840 - val_precision: 0.3553 - val_recall: 0.0155
Epoch 5/260
327/327 [==============================] - 103s 315ms/step - loss: 1.3

KeyboardInterrupt: 

In [9]:
from keras.models import load_model

# Assuming your model is already defined and trained
# (Include the code for model definition, training, etc. before this point)

# Save the entire model to a single file
model.save('faceConfidence2.h5')

/home/jegathees5555/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [1]:
# import cv2
# import numpy as np
# from keras.models import load_model

# # Load the pre-trained model
# loaded_model = load_model('faceConfidence1.h5')  # Replace with the actual path

# # Open the default camera (usually the built-in webcam)
# cap = cv2.VideoCapture()
# img_height = 100
# img_width = 100
# # Check if the camera is opened successfully
# if not cap.isOpened():
#     print("Error: Could not open camera.")
#     exit()

# while True:
#     # Capture a single frame
#     ret, frame = cap.read()

#     # Check if the frame was captured successfully
#     if not ret:
#         print("Error: Could not capture frame.")
#         break

#     # Preprocess the frame (resize, normalize, etc.)
#     processed_frame = cv2.resize(frame, (img_height, img_width))
#     processed_frame = processed_frame / 255.0
#     processed_frame = np.expand_dims(processed_frame, axis=0)

#     # Make predictions using the loaded model
#     predictions = loaded_model.predict(processed_frame)

#     # Interpret the predictions (for example, if it's a classification task)
#     predicted_class = np.argmax(predictions)

#     # Display the frame with prediction information
#     cv2.putText(frame, f"Predicted Class: {predicted_class}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
#     cv2.imshow("Real-time Prediction", frame)

#     # Break the loop if the 'q' key is pressed
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the camera
# cap.release()

# # Close all OpenCV windows
# cv2.destroyAllWindows()


2024-01-22 13:46:27.024280: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-22 13:46:27.059840: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-22 13:46:27.375348: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-22 13:46:27.376940: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-22 13:46:28.584291: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Error: Could not open camera.
Error: Could not capture frame.


: 